<a href="https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/005-EVI-with-Overloaded-Operators-Sentinel-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Overloaded Operators to Compute the EVI (Sentinel-2)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Documentation: [https://eemont.readthedocs.io/en/0.1.7/index.html](https://eemont.readthedocs.io/en/0.1.7/index.html)

## Let's start!

If required, please uncomment:

In [ ]:
#!pip install eemont
#!pip install geemap

Import the required packges.

In [ ]:
import ee, eemont, datetime, geemap

Authenticate and Initialize Earth Engine and geemap.

In [ ]:
Map = geemap.Map()

Point of interest.

In [ ]:
point = ee.Geometry.Point([-76.0269,2.92846])

Get, filter, mask clouds and scale the image collection.

In [ ]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
      .maskClouds()
      .scale()
      .index('EVI')) # Let's compute the EVI using the index() method for comparison

Let's select the required bands for EVI:

In [ ]:
N = S2.select('B8')
R = S2.select('B4')
B = S2.select('B2')

## Overloaded Operators

`eemont` has overloaded the binary operators, rich comparisons and unary operators in the following list for the `ee.Image` class:

(+, -, \*\, /, //, %, \**\, <<, >>, &, |, <, <=, ==, !=, >, >=, -, ~)

Therefore, you can now use them for image operations!

In [ ]:
EVI = 2.5 * (N - R) / (N + 6.0 * R - 7.5 * B + 1.0)

## Visualization

Let's define the EVI visualization parameters:

In [ ]:
visEVI = {
    'min':0,
    'max':1,
    'palette':[
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000',
        '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301'
    ]    
}

And also the RGB visualization parameters:

In [ ]:
visRGB = {
    'min':0,
    'max':0.3,
    'bands':['B4', 'B3', 'B2']
}

Use `geemap` to display results:

In [ ]:
Map.addLayer(S2,visRGB,'RGB')
Map.addLayer(EVI,visEVI,'EVI With Overloaded Operators')
Map.addLayer(S2.select('EVI'),visEVI,'EVI With index() Method')
Map.add_colorbar(visEVI['palette'], caption = 'EVI')
Map.centerObject(point)
Map.addLayerControl()
Map